## ***Machine Learning Products (MLOps)***

#### En este cuaderno jupiter se encuentra todo el proceso del Machine Learning, desde la conexión a la base de datos hasta el sistema de recomendación


#### 1. **Importamos librerias necesarias**

In [97]:
# Importamos librerias 
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os 
import env

#### 2. **Conectamos con la base de datos**

In [ ]:
# Si existen cambios en la credenciales, refrescamos el env
import importlib
import env

importlib.reload(env)

<module 'env' from 'd:\\Personal\\Documentos\\Courses\\Henry\\Henry Students\\Labs\\Cohorte 25 ft\\PG_FT_25\\Proyecto_final_Henry\\Machine_Learning\\env.py'>

In [93]:
# credenciales del AWS

HOST_DB = env.DB_HOST
USER_DB = env.DB_USER
PASS_DB = env.DB_PASS
NAME_DB = env.DB_NAME

In [103]:
# Conectamos con la base de datos 

def BD_connection(host, user_db, user_pass, name_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user_db,
            passwd=user_pass,
            database=name_db,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [96]:
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)

MYSQL DATABASE connection succesful


----------------------------------------------------

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from dotenv import load_dotenv
import os

# 🌟 Cargar variables de entorno
load_dotenv()

# 🔐 Configuración de la conexión a la base de datos
DB_CONFIG = {
    'host': os.getenv("DB_HOST"),
    'port': int(os.getenv("DB_PORT", 3306)),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'database': os.getenv("DB_NAME")
}

# Crear el motor de SQLAlchemy
engine = create_engine(
    f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# Leer los datos desde la tabla enriched_taxi_data
query = "SELECT * FROM enriched_taxi_data"
df = pd.read_sql(query, engine)

# Seleccionar las características y la variable objetivo
X = df[['PULocationID', 'pickup_borough', 'pickup_day', 'pickup_hour']]
y = df['trip_count']

# Convertir variable categórica 'pickup_borough' en variables dummy
X = pd.get_dummies(X, columns=['pickup_borough'], drop_first=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

# Función para predecir la demanda dada una zona, día y hora
def predict_demand(pulocationid, pickup_day, pickup_hour, pickup_borough):
    # Crear un DataFrame con las mismas columnas que el DataFrame original utilizado para entrenar el modelo
    input_data = pd.DataFrame({
        'PULocationID': [pulocationid],
        'pickup_day': [pickup_day],
        'pickup_hour': [pickup_hour]
    })

    # Añadir columnas dummy para 'pickup_borough'
    borough_columns = [col for col in X.columns if col.startswith('pickup_borough_')]
    for col in borough_columns:
        input_data[col] = 0
    
    # Establecer el valor adecuado de la columna dummy correspondiente al 'pickup_borough'
    if f'pickup_borough_{pickup_borough}' in input_data.columns:
        input_data[f'pickup_borough_{pickup_borough}'] = 1
    
    prediction = model.predict(input_data)
    return prediction[0]

# Ejemplo de uso de la función predict_demand
predicted_demand = predict_demand(11, 11, 13, 'Manhattan')
print(f"Predicted Demand: {predicted_demand:.2f}")
